# Loss of Permanent Snow and Ice
In this example, we'll use `sankee` to visualize the conversion from snow and ice to barren and grassland on Baranof Island in Sitka, Alaska, as identified by a time series of MODIS Land Cover imagery between 2001 and 2019. The `geemap` package will allow us to interactively explore the imagery.

## Setup

In [ ]:
import sankee
import ee
import geemap

ee.Authenticate()
ee.Initialize()

In [ ]:
# Create and display the map
Map = geemap.Map()
Map

## Classified Time Series

`sankee` plots are based on changes sampled from time series of classified imagery. To start, we'll load and explore some classified images from MODIS Land Cover.

### Select the Time Series Images

In [ ]:
# Select a pre-made dataset
dataset = sankee.datasets.MODIS_LC_TYPE1

# Select images that belong to the dataset. You can run dataset.get_images() to preview a list of image IDs. 
start = ee.Image('MODIS/006/MCD12Q1/2001_01_01')
mid = ee.Image('MODIS/006/MCD12Q1/2010_01_01')
end = ee.Image('MODIS/006/MCD12Q1/2019_01_01')

# Collect the images into a list in chronological order
image_list = [start, mid, end]

### Show the Images

Visualizing classified imagery can help to identify areas that experienced land cover change. We'll add them to the map so that we can interactively explore them.

In this example, we use properties of the dataset to set the visualization parameters of the images. This is possible for MODIS because the class IDs are sequential, but won't work with all datasets.

In [ ]:
# Set the visualization parameters
min_val = min(dataset.keys)
max_val = max(dataset.keys)
palette = list(dataset.palette.values())

# Add the images to the map
Map.addLayer(start.select(dataset.band), {"min": min_val, "max": max_val, "palette": palette}, "2001")
Map.addLayer(mid.select(dataset.band), {"min": min_val, "max": max_val, "palette": palette}, "2010")
Map.addLayer(end.select(dataset.band), {"min": min_val, "max": max_val, "palette": palette}, "2019")

## Area of Interest
`sankee` plots are sampled within one area of interest to visualize changes in that area. By exploring the imagery we added above and toggling between the imagery dates, we can find areas that experienced change. Below, we'll zoom in on Baranof Island where substantial loss of snow and ice is visible over the time series.

### Interactively
The area of interest can be defined interactively using the `geemap` drawing tools. You can experiment by drawing a polygon on the map and running the code below to set the last drawn feature as the area of interest.

In [ ]:
# aoi = Map.draw_last_feature.geometry()

### Manually
For repeatability, we'll manually define our area of interest.

In [ ]:
aoi = ee.Geometry.Polygon([
    [-135.197519, 56.888425],
    [-134.774492, 56.888425],
    [-134.774492, 57.240839],
    [-135.197519, 57.240839],
])

### Show the Area of Interest

In [ ]:
Map.addLayer(aoi, {"opacity": 0.2}, "AOI")
Map.centerObject(aoi, zoom=9)

## Sankey Plot

Now that we have a classified time series and an area of interest, we can generate a plot that shows how land cover changed over time.

### Generating the Default Plot

In [ ]:
# Generate and display the plot.
plot = sankee.sankify(image_list, aoi, dataset=dataset)
plot

### Refining the Plot

The current plot is messy and hard to interpret. We can refine it by reducing the number of classes shown and excluding irrelevant classes.

In [ ]:
# Setting max classes removes classes with few observations
max_classes = 3

# Evergreen conifer forest (1) and Mixed forest (5) have a lot of observations but don't experience
# much change, so we'll remove them from the plot. If you're not sure which ID corresponds to which
# class, you can run dataset.df.
exclude_classes = [1, 5]

# The resulting plot clearly shows how permanent snow and ice is being converted to grassland and barren.
plot = sankee.sankify(image_list, aoi, dataset=dataset, max_classes=max_classes, exclude=exclude_classes)
plot